In [17]:
%%capture
from pathlib import Path
from zipfile import ZipFile

import pandas as pd
import clarify
import requests
from io import BytesIO as ioDuder
from lxml import etree
from selenium import webdriver


def link_to_xml(link):
    s = clarify.Jurisdiction(url=link, level='county')
    r = requests.get(s.report_url('xml'), stream=True)
    z = ZipFile(ioDuder(r.content))
    return z.open('detail.xml')


driver = webdriver.Chrome()
back_link = 'http://results.enr.clarityelections.com/GA/63991/184321/en/select-county.html'
# I don't think driver.back() actually works for clarity elections websites.
driver.get(back_link)

# selenium doesn't like multiple windows

# Get county names for the current state
county_dict = {county.text: county for county in driver.find_elements_by_tag_name("li")}




# For dev use 1 county
county = 'Cherokee'


driver.execute_script("arguments[0].scrollIntoView();", county_dict[county])


county_dict[county].click()


window_before = driver.window_handles[0]


window_after = driver.window_handles[1]


driver.switch_to_window(window_after)

returns = link_to_xml(link=driver.current_url).read()
root = etree.XML(returns)

# Write to ./returns.xml for manual inspection
xml_path = Path('.', 'returns.xml').resolve()
xml_path.write_text(etree.tostring(root).decode('utf-8'))

returns = {}
for contest in root.xpath('Contest'):
    contest_name = contest.get('text')
    print(contest_name)
    # Create keys named for contests, e.g.  "HOLLY SPRINGS CITY COUNCIL W5"
    returns[contest_name] = {}
    for choice in contest.xpath('Choice'):
        # Choices represent candidates
        candidate = choice.get('text')
        returns[contest_name][candidate] = {}
        for vote_type in choice.xpath('VoteType'):
            # Vote types are Election-Day, mail, etc.
            vote_type_name = vote_type.get('name')
            returns[contest_name][candidate][vote_type_name] = {'precincts': {}}
#            for county in vote_type.xpath('County'):
#                county_name = county.get('name')
#                returns[contest_name][candidate][vote_type_name]['counties'][county_name] = county.get('votes')
            for precinct in vote_type.xpath('Precinct'):
                precinct_name = precinct.get('name')
                returns[contest_name][candidate][vote_type_name]['precincts'][precinct_name] = precinct.get('votes')
                

            
                
                
                


In [51]:
df=pd.DataFrame()

for office in returns.keys():
    offices = returns[office]
    for candidate in offices:
        candidates = offices[candidate]
        for vote_type in candidates:
            vote_types = candidates[vote_type]
            for precinct in vote_types:
                tmp=pd.DataFrame()
                tmp=tmp.from_dict(vote_types[precinct], orient='index')
                tmp['office']=office
                tmp['vote_type']=vote_type
                tmp['candidate']=candidate
                tmp['county']=county
                df = df.append(tmp)
                #print(tmp)
        

In [52]:
df.to_csv('~/test_output.csv')